In [1]:
# 한글 문장 데이터
sentences = [
    '안녕하세요, 반갑습니다.',
    '오늘 날씨가 좋네요.',
    '내일은 비가 올 것 같아요.',
    '저는 공부를 좋아합니다.'
    '저는 공부를 좋아합니다.'
]

# 각 문장에 대한 레이블
categories = ['인사', '날씨', '날씨', '취미']


# 추가 연습 예제
sentences = [
    "오늘 날씨가 좋아서 나들이 가고 싶다.",
    "이 영화는 정말 재미있었어요.",
    "맛있는 음식을 먹으러 갈까요?",
    "운동을 하면 건강에 좋아지는 것 같아요.",
    "공부하기 싫어서 미루고 있어요.",
    "여행 계획을 세우고 있는데 어디로 갈까요?",
    "좋은 책을 읽으면 마음이 편안해져요.",
    "오늘은 친구들과 만나서 재미있게 놀았어요.",
    "새로운 언어를 배우는 것은 어려워도 흥미로워요.",
    "주말에 가족들과 함께 시간을 보내기로 했습니다."
]

categories = [
    "여행/레저",
    "엔터테인먼트/영화",
    "식음료/레스토랑",
    "건강/운동",
    "교육/학습",
	"여행/계획",
	"엔터테인먼트/도서",
	"인간관계/친구",
	"교육/언어학습",
	"가족 / 가정"
]

## Vectorizer

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from konlpy.tag import Okt

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# 불용어 리스트 생성 (예시)
stopwords = ['가', '고', '을', '를', '이', '는']

# 토크나이징 함수 정의
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

# Vectorizer 설정 및 fit_transform 실행 
vectorizer = TfidfVectorizer(tokenizer=tokenizer)
X = vectorizer.fit_transform(sentences)
# type(X), X.toarray()

## LDA 학습

In [3]:
# LDA 모델 설정 및 fit 실행 
n_topic=2
lda_model=LatentDirichletAllocation(n_components=n_topic, learning_method='online', random_state=777, max_iter=1)
lda_top=lda_model.fit(X)
print(lda_model.components_)

[[0.93256387 0.86521185 1.03883671 1.02523653 0.80913581 1.07368176
  1.0150056  1.13778466 0.92450749 0.88634471 1.05362286 0.9507389
  1.02789763 0.91746244 1.15322294 0.98473815 1.01458403 0.97312496
  1.02485685 0.83306221 0.8983354  1.04070562 1.15448415 0.94935007
  0.81380718 0.83171438 1.05212879 0.94951346 0.774172   1.00437099
  1.06190695]
 [0.87558689 0.97806064 1.1132956  1.00760608 0.92550612 0.97235507
  0.95750357 0.91323946 0.89910456 1.05202651 0.96084095 1.04561235
  0.91227673 0.95515443 1.13116196 1.02906899 1.02264529 0.97120065
  0.87476442 0.91726269 0.82893472 0.92970834 0.8633624  0.95138158
  1.09759723 1.05571554 0.99581933 0.92367191 0.93521605 0.90925151
  0.94643439]]


In [4]:
lda_model.bound_, lda_model.topic_word_prior_, lda_model.doc_topic_prior_

(115.22757889932885, 0.5, 0.5)

In [5]:
# print(lda_model.transform(X))

## 각 토픽에 대해 가장 높은 확률을 가진 단어들과 각 문장

In [6]:
import pandas as pd
import numpy as np

# 각 문장이 어느 토픽에 속하는지 확인
topic_output = lda_model.transform(X)

# dataframe으로 변환
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
docnames = ["Doc" + str(i) for i in range(len(sentences))]
df_document_topic = pd.DataFrame(np.round(topic_output, 2), columns=topicnames, index=docnames)

# dominant topic 컬럼 생성
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

print(df_document_topic)

      Topic0  Topic1  dominant_topic
Doc0    0.74    0.26               0
Doc1    0.63    0.37               0
Doc2    0.28    0.72               1
Doc3    0.60    0.40               0
Doc4    0.69    0.31               0
Doc5    0.49    0.51               1
Doc6    0.29    0.71               1
Doc7    0.72    0.28               0
Doc8    0.37    0.63               1
Doc9    0.33    0.67               1


## 각 문서에 대해 가장 확률이 높은 토픽과 그 토픽의 상위 단어들을 함께 출력

In [7]:
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words["Topic" + str(topic_idx)] = " ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])
    return top_words

# 상위 단어 추출
n_top_words = 5
top_words = get_top_words(lda_model, vectorizer.get_feature_names(), n_top_words)

# DataFrame 생성
df_topic_keywords = pd.DataFrame(top_words.items(), columns=['Topic', 'TopWords'])

# df_document_topic와 병합
df_merged = pd.merge(df_document_topic.reset_index(), df_topic_keywords, left_on='dominant_topic', right_on=df_topic_keywords.index)
df_merged.set_index('index', inplace=True)

print(df_merged)

       Topic0  Topic1  dominant_topic   Topic         TopWords
index                                                         
Doc0     0.74    0.26               0  Topic0  오늘 보내다 날씨 공부 하다
Doc1     0.63    0.37               0  Topic0  오늘 보내다 날씨 공부 하다
Doc3     0.60    0.40               0  Topic0  오늘 보내다 날씨 공부 하다
Doc4     0.69    0.31               0  Topic0  오늘 보내다 날씨 공부 하다
Doc7     0.72    0.28               0  Topic0  오늘 보내다 날씨 공부 하다
Doc2     0.28    0.72               1  Topic1  보내다 갈다 음식 읽다 마음
Doc5     0.49    0.51               1  Topic1  보내다 갈다 음식 읽다 마음
Doc6     0.29    0.71               1  Topic1  보내다 갈다 음식 읽다 마음
Doc8     0.37    0.63               1  Topic1  보내다 갈다 음식 읽다 마음
Doc9     0.33    0.67               1  Topic1  보내다 갈다 음식 읽다 마음


In [8]:
# 토픽별로 중요도가 높은 단어들을 출력
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


# n_top_words = 10 # 상위 10개의 단어를 출력하도록 설정 
n_top_words = 2
print_top_words(lda_model, vectorizer.get_feature_names(), n_top_words)


Topic #0:
오늘 보내다

Topic #1:
보내다 갈다

